In [2]:
%matplotlib widget

import abc
import torch.nn.modules.module
from torch.export import export, Dim
from torch.export.exported_program import ExportedProgram, InputKind, OutputKind
import uuid
import torch._dynamo
from torch._functorch.aot_autograd import aot_module, aot_module_simplified
from torch._dynamo.backends.common import aot_autograd
from functorch.compile import make_boxed_func
from torch._decomp import core_aten_decompositions
from torch._dynamo.backends.inductor import inductor
from torch._subclasses.fake_tensor import FakeTensorMode
from torch._guards import detect_fake_mode
from typing import *
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import functools
import math
import random


from collections import namedtuple
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn, optim
import torch.nn.functional as F
import torch
import functools
import os

In [44]:
import functools
import inspect
import contextlib
from torch.utils._python_dispatch import TorchDispatchMode
from torch.overrides import enable_reentrant_dispatch
from collections import defaultdict
from torch._subclasses.fake_tensor import FakeTensorMode
from torch.utils._pytree import tree_map
from typeguard import typechecked
from multimethod import multimethod
from torch.overrides import TorchFunctionMode, resolve_name
from torch.utils._python_dispatch import TorchDispatchMode

ENABLE_NORM_DISPATCH = True

@contextlib.contextmanager
def disable_norm_dispatch():
    global ENABLE_NORM_DISPATCH
    old_flag = ENABLE_NORM_DISPATCH
    ENABLE_NORM_DISPATCH = False
    yield
    ENABLE_NORM_DISPATCH = old_flag

HANDLED_FUNCTIONS: Dict[Callable, Callable] = {}


def get_output_fake_tensors(func, *args, **kwargs):
    # Create a fake mode
    fake_mode = FakeTensorMode(allow_non_fake_inputs=True)
    def convert_from_real_tensor(x):
        if isinstance(x, torch.Tensor):
            return fake_mode.fake_tensor_converter.from_real_tensor(fake_mode, x)
        return x
    # Fakeify some real tensors
    with fake_mode, disable_norm_dispatch():
        args = tree_map(convert_from_real_tensor, args)
        kwargs = tree_map(convert_from_real_tensor, kwargs)
        return func(*args, **kwargs)


def implements(torch_function):
    """Register a torch function override for ScalarTensor"""
    def decorator(func):
        # sig = inspect.signature(func)
        # func = typechecked(func)
        functools.update_wrapper(func, torch_function)
        assert torch_function not in HANDLED_FUNCTIONS
        HANDLED_FUNCTIONS[torch_function] = func
        return func
    return decorator



class FunctionLog(TorchFunctionMode):
    def __torch_function__(self, func, types, args, kwargs=None):
        if ENABLE_NORM_DISPATCH:
            print(f"Function Log: {func}(*{args}, **{kwargs})")
            if any(issubclass(t, NormTensorBase) for t in types) and func in HANDLED_FUNCTIONS:
                return NotImplemented
        return func(*args, **(kwargs or {}))

class DispatchLog(TorchDispatchMode):
    def __torch_dispatch__(self, func, types, args, kwargs=None):
        if ENABLE_NORM_DISPATCH:
            print(f"Dispatch Log: {func}(*{args}, **{kwargs})")
            if any(issubclass(t, NormTensorBase) for t in types):
                return NotImplemented
        return func(*args, **(kwargs or {}))


class NormTensorBase(torch.Tensor):
    # a tensor container

    @staticmethod
    def __new__(cls, norm_size: Union[float, torch.Tensor], *, size: torch.Size,
                dtype: torch.dtype, device: torch.device, requires_grad=None):
        return cls._make_wrapper_subclass(cls, size, dtype=dtype, device=device, requires_grad=False)

    def __init__(self, norm_size: Union[float, torch.Tensor], *, size: torch.Size,
                 dtype: torch.dtype, device: torch.device, requires_grad=None):
        if isinstance(norm_size, torch.Tensor):
            assert requires_grad is None
            self._norm_size = norm_size
        else:
            self._norm_size = torch.full((), norm_size, dtype=torch.float32, requires_grad=requires_grad)

    @classmethod
    def from_tensor(cls, norm_size: Union[float, torch.Tensor], tensor: torch.Tensor, *, requires_grad=None):
        return cls(norm_size, size=tensor.size(), dtype=tensor.dtype, device=tensor.device, requires_grad=requires_grad)

    @property
    def norm_size(self) -> torch.Tensor:
        return self._norm_size

    def __repr__(self):
        return f"{self.__class__.__name__}(norm_size={self.norm_size!r})"

    @classmethod
    def __torch_function__(cls, func, types, args=(), kwargs=None):
        if kwargs is None:
            kwargs = {}
        if ENABLE_NORM_DISPATCH and func in HANDLED_FUNCTIONS:
            print(f"cls Function Log: {func}(*{args}, **{kwargs})")
            with disable_norm_dispatch():
                return HANDLED_FUNCTIONS[func](*args, **kwargs)
        # for handler, sig in HANDLED_FUNCTIONS.get(func, []):
        #     print(f"Trying {handler}", sig, args, kwargs)
        #     try:
        #         bound = sig.bind(*args, **kwargs)
        #     except TypeError as e:
        #         continue
        #     with enable_reentrant_dispatch():
        #         out = handler(*bound.args, **bound.kwargs)
        #     print(out.norm_size.__class__)
        #     print(out)
        #     print(out.norm_size)
        #     return out
        return super().__torch_function__(func, types, args, kwargs)

    @classmethod
    def __torch_dispatch__(cls, func, types, args=(), kwargs=None):
        if kwargs is None:
            kwargs = {}
        return NotImplemented
        if ENABLE_NORM_DISPATCH and func in HANDLED_FUNCTIONS:
            print(f"cls Dispatch Log: {func}(*{args}, **{kwargs})")
            with enable_reentrant_dispatch():
                return HANDLED_FUNCTIONS[func](*args, **kwargs)
        # for handler, sig in HANDLED_FUNCTIONS.get(func, []):
        #     print(f"Trying {handler}", sig, args, kwargs)
        #     try:
        #         bound = sig.bind(*args, **kwargs)
        #     except TypeError as e:
        #         continue
        #     with enable_reentrant_dispatch():
        #         out = handler(*bound.args, **bound.kwargs)
        #     print(out.norm_size.__class__)
        #     print(out)
        #     print(out.norm_size)
        #     return out
        return NotImplemented


class RMS_NormTensor(NormTensorBase):
    pass

class RMS_RMS_NormTensor(NormTensorBase):
    pass

class L1_NormTensor(NormTensorBase):
    pass


@implements(torch.ops.aten.linear.default)
@implements(torch.nn.functional.linear)
@multimethod
def linear(input: RMS_NormTensor, weight: RMS_RMS_NormTensor, bias: Optional[RMS_NormTensor] = None) -> RMS_NormTensor:
    final_norm_size = input.norm_size * weight.norm_size
    if bias is not None:
        final_norm_size += bias.norm_size
    out_fake = get_output_fake_tensors(torch.nn.functional.linear, input, weight, bias=bias)
    return RMS_NormTensor(final_norm_size, size=out_fake.size(), dtype=out_fake.dtype, device=out_fake.device)


# @implements(torch.ops.aten.unsqueeze.default)
# @multimethod
# def unsqueeze(input: RMS_NormTensor, dim: int) -> RMS_NormTensor:
#     out_fake = get_output_fake_tensors(torch.ops.aten.unsqueeze.default, input, dim)
#     return RMS_NormTensor(input.norm_size, size=out_fake.size(), dtype=out_fake.dtype, device=out_fake.device)

# @implements(torch.ops.aten.squeeze_.dim)
# @multimethod
# def squeeze(input: RMS_NormTensor, dim: int) -> RMS_NormTensor:
#     out_fake = get_output_fake_tensors(torch.ops.aten.squeeze_.dim, input, dim)
#     return RMS_NormTensor(input.norm_size, size=out_fake.size(), dtype=out_fake.dtype, device=out_fake.device)

# @implements(torch.ops.aten.t.default)
# @multimethod
# def t(input: RMS_RMS_NormTensor) -> RMS_RMS_NormTensor:
#     assert input.ndim == 2
#     return RMS_RMS_NormTensor(input.norm_size, size=input.size()[::-1], dtype=input.dtype, device=input.device)

# # @t.register
# # def t(input: RMSNormTensor) -> RMSRMSNormTensor:
# #     assert input.ndim == 2
# #     print(input.size(), 'l')
# #     return RMSRMSNormTensor(input.norm_size + 2, size=input.size()[::-1], dtype=input.dtype, device=input.device)

# @implements(torch.ops.aten.addmm.default)
# @multimethod
# def addmm(input: RMS_NormTensor, mat1: RMS_NormTensor, mat2: RMS_RMS_NormTensor, *, beta: float = 1, alpha: float = 1) -> RMS_NormTensor:
#     # output = input * beta + mat1 @ mat2 * alpha
#     final_norm_size = input.norm_size * beta + mat1.norm_size * mat2.norm_size * alpha
#     out_fake = get_output_fake_tensors(torch.ops.aten.addmm.default, input, mat1, mat2, beta=beta, alpha=alpha)
#     return RMS_NormTensor(final_norm_size, size=out_fake.size(), dtype=out_fake.dtype, device=out_fake.device)

# @addmm.register
# def _(input: RMS_NormTensor, mat1: RMS_NormTensor, mat2: RMS_RMS_NormTensor, *, beta: float = 1, alpha: float = 1) -> RMS_NormTensor:
#     # output = input * beta + mat1 @ mat2 * alpha
#     final_norm_size = input.norm_size * beta + mat1.norm_size * mat2.norm_size * alpha
#     out_fake = get_output_fake_tensors(torch.ops.aten.addmm.default, input, mat1, mat2, beta=beta, alpha=alpha)
#     return RMS_NormTensor(final_norm_size, size=out_fake.size(), dtype=out_fake.dtype, device=out_fake.device)


# @implements(torch.ops.aten.mm.default)
# @multimethod
# def mm(input: RMS_NormTensor, mat1: RMS_RMS_NormTensor) -> RMS_NormTensor:
#     final_norm_size = input.norm_size * mat1.norm_size
#     out_fake = get_output_fake_tensors(torch.ops.aten.mm.default, input, mat1)
#     return RMS_NormTensor(final_norm_size, size=out_fake.size(), dtype=out_fake.dtype, device=out_fake.device)


In [147]:
torch.export.export(
    nn.Linear(15, 16),
    (torch.randn(2, 15, requires_grad=True),)
).graph.print_tabular()

opcode         name         target               args                             kwargs
-------------  -----------  -------------------  -------------------------------  --------
placeholder    p_fn_weight  p_fn_weight          ()                               {}
placeholder    p_fn_bias    p_fn_bias            ()                               {}
placeholder    input        input                ()                               {}
call_function  linear       aten.linear.default  (input, p_fn_weight, p_fn_bias)  {}
output         output       output               ((linear,),)                     {}


In [66]:
torch.__version__

'2.5.1'

In [78]:
if torch.__version__ > '2.5.1':
    def default_decompositions():
        return torch.export.default_decompositions()
else:
    from torch._decomp import core_aten_decompositions
    def default_decompositions():
        return core_aten_decompositions()


decomps = default_decompositions()

In [104]:
for op_overload, decomp in decomps.items():
    if 'linear' in str(op_overload):
        print(op_overload)
        print(decomp)


aten.upsample_linear1d.default
<function upsample_linear1d at 0x13074a520>
aten.upsample_linear1d.out
<function upsample_linear1d at 0x13074a520>
aten.upsample_bilinear2d.vec
<function _upsample_linear_vec at 0x13074a5c0>
aten.upsample_bilinear2d.default
<function upsample_bilinear2d at 0x13074a8e0>
aten.upsample_bilinear2d.out
<function upsample_bilinear2d at 0x13074a8e0>
aten.upsample_trilinear3d.vec
<function _upsample_linear_vec at 0x13074a5c0>
aten.upsample_trilinear3d.default
<function upsample_trilinear3d at 0x13074aac0>
aten.upsample_trilinear3d.out
<function upsample_trilinear3d at 0x13074aac0>


In [100]:
next(iter(decomps.keys()))._opname

'addcdiv'

In [98]:
dir(next(iter(decomps.keys())))

['__annotations__',
 '__call__',
 '__class__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_can_decompose',
 '_defined_in_python',
 '_dispatch_cache',
 '_get_dispatch',
 '_handle',
 '_lazy_handle',
 '_name',
 '_namespace',
 '_op',
 '_op_dk',
 '_opname',
 '_overloadname',
 '_overloadpacket',
 '_schema',
 '_tags',
 '_uncache_dispatch',
 'decompose',
 'functorch_table',
 'has_kernel_for_any_dispatch_key',
 'has_kernel_for_dispatch_key',
 'is_view',
 'name',
 'namespace',
 'op',
 'overloadpacket',
 'py_functionalize_impl',
 'py_impl',
 'py_kernels',
 'python_key_table',
 'redispatch',
 'tags']

In [105]:
from torch._functorch.aot_autograd import aot_module_simplified

ep = torch.export.export(
    nn.Linear(15, 16),
    (torch.randn(2, 15, requires_grad=True),)
)

# ep = ep.run_decompositions()

gm = ep.module()

# gm = aot_module(gm, (torch.randn(2, 15, requires_grad=True),))


In [106]:
gm

GraphModule()

In [107]:
print(gm)

GraphModule()



def forward(self, input):
    input, = fx_pytree.tree_flatten_spec(([input], {}), self._in_spec)
    weight = self.weight
    bias = self.bias
    input_1 = input
    linear = torch.ops.aten.linear.default(input_1, weight, bias);  input_1 = weight = bias = None
    return pytree.tree_unflatten((linear,), self._out_spec)
    
# To see more debug info, please use `graph_module.print_readable()`


In [49]:
with FunctionLog():
    object.__setattr__(gm, 'weight', RMS_RMS_NormTensor.from_tensor(4, gm.weight, requires_grad=True))
    object.__setattr__(gm, 'bias', RMS_NormTensor.from_tensor(5, gm.bias, requires_grad=True))
    y = gm(
        RMS_NormTensor.from_tensor(3, torch.randn(2, 15, requires_grad=True)),
    )
    print(y, y.norm_size)

Function Log: <method 'size' of 'torch._C.TensorBase' objects>(*(Parameter containing:
tensor([[ 2.5037e-01, -2.3509e-01,  6.9403e-02,  1.5359e-01,  2.1446e-01,
          5.0863e-02, -1.3417e-01,  2.2040e-01, -3.1974e-02, -1.8737e-01,
         -6.5651e-02,  2.3970e-01,  2.1037e-01,  8.7696e-02, -3.6649e-02],
        [-1.6843e-01,  2.1243e-01, -2.1920e-01,  8.5752e-02, -2.2146e-01,
          2.3418e-01,  1.7371e-01, -2.2969e-01,  1.9210e-02, -3.2548e-02,
          1.6985e-01, -5.8576e-02, -9.1809e-02, -2.4581e-01,  1.3280e-01],
        [ 1.4920e-01, -4.6759e-02, -2.2853e-02,  1.2846e-01,  1.4195e-01,
         -8.8691e-02, -6.5738e-02, -1.0860e-01, -7.6861e-02,  8.1323e-02,
         -2.2642e-01,  8.6144e-02, -1.0688e-01,  1.4019e-01,  1.7083e-02],
        [ 1.6495e-01,  2.1578e-01,  1.8354e-01,  2.5663e-01,  1.5602e-01,
          1.6883e-01,  7.8164e-02,  1.2637e-01,  7.6798e-02,  9.3937e-02,
         -6.5100e-02,  4.1794e-02, -1.6774e-01, -2.1921e-01,  2.3303e-01],
        [-3.3647e-02,

In [50]:
torch.autograd.grad(y.norm_size, [gm.weight.norm_size, gm.bias.norm_size])

(tensor(3.), tensor(1.))

In [36]:
with FunctionLog():
    # torch.randn(1, 15, requires_grad=True)
    y = torch.nn.functional.linear(
        RMS_NormTensor(3, size=(1, 15), dtype=torch.float32, device=torch.device("cpu"), requires_grad=True),
        RMS_RMS_NormTensor(4, size=(16, 15), dtype=torch.float32, device=torch.device("cpu"), requires_grad=True),
        RMS_NormTensor(5, size=(16,), dtype=torch.float32, device=torch.device("cpu"), requires_grad=True)
    )
    print(y, y.norm_size)


Function Log: <class 'torch.device'>(*('cpu',), **None)
Function Log: <built-in method full of type object at 0x124604008>(*((), 3), **{'dtype': torch.float32, 'requires_grad': True})
Function Log: <class 'torch.device'>(*('cpu',), **None)
Function Log: <built-in method full of type object at 0x124604008>(*((), 4), **{'dtype': torch.float32, 'requires_grad': True})
Function Log: <class 'torch.device'>(*('cpu',), **None)
Function Log: <built-in method full of type object at 0x124604008>(*((), 5), **{'dtype': torch.float32, 'requires_grad': True})
Function Log: <built-in function linear>(*(RMS_NormTensor(norm_size=tensor(3., requires_grad=True)), RMS_RMS_NormTensor(norm_size=tensor(4., requires_grad=True)), RMS_NormTensor(norm_size=tensor(5., requires_grad=True))), **None)
Function Log: <function Tensor.__repr__ at 0x122611bc0>(*(tensor(3., requires_grad=True),), **{'tensor_contents': None})
Function Log: <function Tensor.__repr__ at 0x122611bc0>(*(tensor(4., requires_grad=True),), **{'t

In [127]:
y = (x:=RMSNormTensor(3, requires_grad=True)) + RMSRMSNormTensor(4)
print(y, y.norm_size)

Dispatch Log: aten.add.Tensor(*(RMSNormTensor(norm_size=3, requires_grad=False), RMSRMSNormTensor(norm_size=4, requires_grad=False)), **{})
tensor(7., grad_fn=<AddBackward0>)
z: RMSNormTensor(norm_size=7, requires_grad=False) tensor(7., grad_fn=<AddBackward0>) [tensor(3., requires_grad=True), tensor(4.)] tensor(7., grad_fn=<AddBackward0>)
RMSNormTensor(norm_size=7, requires_grad=False) tensor(7., grad_fn=<AddBackward0>)


In [96]:
torch.autograd.grad(y.norm_size, [x.norm_size])

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [72]:
y = (x:=RMSNormTensor(3, requires_grad=True)) + RMSRMSNormTensor(4)
print(y, y.norm_size)
torch.autograd.backward([y])
y, x.grad



Dispatch Log: aten.add.Tensor(*(RMSNormTensor(norm_size=3.0), RMSRMSNormTensor(norm_size=4.0)), **{})
RMSNormTensor(norm_size=7.0) tensor(7.)
Dispatch Log: aten.ones_like.default(*(RMSNormTensor(norm_size=7.0),), **{'pin_memory': False, 'memory_format': torch.preserve_format})


ZeroDivisionError: division by zero

In [60]:
y%debu

In [33]:

torch.ops.aten.linear(torch.randn(10, 10), torch.randn(10, 10), torch.randn(10))

tensor([[-2.9366, -2.1385,  0.2499,  0.1769,  3.3125,  0.5611,  0.2110, -0.9544,
         -1.4054, -1.0516],
        [-1.3193,  2.8552, -2.4230,  2.6803,  4.9918,  3.8054, -1.6162, -3.6952,
         -3.6532, -1.4474],
        [-1.9989,  3.0092, -8.1097,  0.5183,  2.0837, -0.9222,  0.3195, -5.3470,
         -3.0957,  2.3705],
        [ 5.2813,  2.8777,  0.0169, -0.6770, -3.8537,  2.0995, -3.8720, -1.3664,
         -2.2780,  2.0936],
        [-0.3890, -0.7412,  2.1590,  0.5803, -0.2978,  0.4573, -0.6891,  0.1847,
          0.4717, -1.0158],
        [-4.2937,  4.5814,  0.6280,  0.9427, -4.7592, -3.3714, -5.1919, -6.9244,
         -0.7811,  4.4841],
        [-3.6631, -4.5173,  1.1486, -3.5798, -0.8339, -1.9358,  3.3141,  1.8897,
          1.1224,  1.5577],
        [ 2.7387, -2.0768,  0.9836, -0.7142,  3.7925, -3.0117, -0.3672, -1.0644,
          4.0226, -2.9036],
        [-3.9206,  0.3760,  3.8509, -0.3813, -1.1643,  0.2692, -2.0817, -1.4092,
         -0.0249,  0.6661],
        [ 7.7065,  